# Main Execution Pipeline

In [ ]:
# -------
# Install
# -------
# !pip install torch torchvision torchmetrics

In [ ]:
import os
import random
import shutil
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.models import resnet50, ResNet50_Weights
from torchmetrics import Accuracy, F1Score
from tqdm.notebook import tqdm
from functions import (
    prepare_validation_set,
    prepare_datasets,
    prepare_dataloaders,
    build_model,
    train_with_validation,
    evaluate_model,
    save_model_formats)
# ---------------------------
# Set random seeds for reproducibility
# ---------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


In [ ]:
# ---------------------------------------
# Workflow: Prepare data, build model, train, evaluate, and save.
# ---------------------------------------

# Step 1: Prepare validation set if not already done
prepare_validation_set()

# Step 2: Prepare datasets and dataloaders
train_dataset, val_dataset, test_dataset = prepare_datasets()
train_loader, val_loader, test_loader = prepare_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size=32
)

# Step 3: Build model
model = build_model()

# Step 4: Set loss, optimizer, scheduler
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# Step 5: Train model
train_with_validation(
    model, train_loader, val_loader, criterion, optimizer, scheduler,
    num_epochs=50, patience=5
)

# Step 6: Load best model
model.load_state_dict(torch.load('model.pth'))

# Step 7: Evaluate on test set
evaluate_model(model, test_loader)

# Step 8: Save model in multiple formats
save_model_formats(model)


